# Try this demo out!
```python
# Install snowflake.demos and run this in your local development environment or in Snowflake Notebooks
import snowflake.demos
snowflake.demos.load_demo('visual-data-stories')
```

# Visual Data Stories with Snowflake Notebooks

In this tutorial, we will walk you through the different ways you can enrich your data narrative through engaging visuals in Snowflake Notebooks. We will demonstrate how you can develop visualizations, work with Markdown text, embed images, and build awesome data apps all within your notebook, alongside your code and data.

**Note**: Before we start, please make sure that you've installed Plotly from the `Packages` drop down in the top right corner. 

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import numpy as np

## Data visualizations 📊

With Snowflake Notebooks, you can use your favorite Python visualization library, including plotly, altair, and matplotlib to develop your visualization.

First, let's generate a toy snowfall dataset.

In [ ]:
# Create the sample dataframe
df = pd.DataFrame({
    "region": (["Sierra Nevada"] * 3 +
               ["Lake Tahoe"] * 3 +
               ["Mammoth"] * 3),
    "month": ["December", "January", "February"] * 3,
    "snowfall_inches": [12.1, 20.2, 15.3, 10.1, 18.7, 12.6, 25.5, 30.0, 20.3]
})

df

## Plotting with Altair

Now let's plot a bar chart in Altair. You can learn more about Altair [here](https://altair-viz.github.io/).

In [ ]:
import altair as alt

# Create a faceted bar chart
alt.Chart(df).mark_bar().encode(
    x=alt.X("month:N", title="Month"),
    y=alt.Y("snowfall_inches:Q", title="Snowfall (inches)"),
    color=alt.Color("month:N", legend=None)
).facet(
    column=alt.Column(
        "region:N",
        title="Region",
    )
).properties(
    title="Snowfall by Region and Month"
)

## Plotting with Plotly

Let's do the same plot with plotly. You can learn more about plotly [here](https://plotly.com/python/plotly-express/).

In [ ]:
import plotly.express as px

# Create a faceted bar chart using Plotly
fig = px.bar(
    df,
    x="month",
    y="snowfall_inches",
    color="month",
    facet_col="region",
    title="Snowfall by Region and Month"
)
fig

## Develop your narrative with Markdown cells

With Snowflake Notebooks, you can leverage Markdown language to develop rich text displays with formatting. 

You can change a cell type to render Markdown by clicking on the dropdown on the top left to select `Markdown` (or use `m` as the hotkey to convert a cell type to Markdown).

# Top-level Header 
Markdown: `# Top-level Header`

## 2nd-level Header 
Markdown: `## 2nd-level Header`

### 3rd-level Header 
Markdown: `### 3rd-level Header`

### Inline Text Formatting

Here is some *italicized text* and **bolded text**. 

Markdown: `Here is some *italicized text* and **bolded text**.`

Here is a link to the [Snowflake website](https://snowflake.com/)

Markdown: `Here is a link to the [Snowflake website](https://snowflake.com/)`

From here on, you can double click onto each Markdown cell to take a look at the underlying Markdown content.

Here is a bulleted list (with emojis 😊)

- ❄️
- ❄️❄️
- ❄️❄️❄️
    - Nested ❄️
    - Nested ❄️❄️

## Formatting code

Python: 
```python
import pandas as pd
df = pd.DataFrame([1,2,3])
```

SQL: 
```sql
SELECT * FROM MYTABLE
```

Inline Code formatting: 

My data is stored in `DB.SCHEMA`. 

## Embedding Images 🖼️


You can use the Markdown Syntax to embed images in your notebook.
```
![](https://www.snowflake.com/wp-content/themes/snowflake/assets/img/brand-guidelines/logo-sno-blue-example.svg)
```
![](https://www.snowflake.com/wp-content/themes/snowflake/assets/img/brand-guidelines/logo-sno-blue-example.svg)


It even works with a GIF animation!

```
![](https://upload.wikimedia.org/wikipedia/commons/2/2c/Rotating_earth_%28large%29.gif)
```
![](https://upload.wikimedia.org/wikipedia/commons/2/2c/Rotating_earth_%28large%29.gif)

## Bring your Notebook alive with Streamlit

You can bring your data narrative alive in notebooks and make it even more interactive by using Streamlit.

[Streamlit](https://streamlit.io/) is an open-source framework for building interactive data apps in Python (not a single line of HTML or Javascript required!)

Unlike in other notebooks where you need to navigate to a separate terminal window to serve up your Streamlit app, you can test and develop your Streamlit app directly in your notebook. 

We saw how you can embed images using Markdown. Here we show how you can embed images in your notebook using Streamlit which gives you more image customization options.

In [ ]:
import streamlit as st
st.image("https://www.snowflake.com/wp-content/themes/snowflake/assets/img/brand-guidelines/logo-sno-blue-example.svg",width=500)

In [ ]:
# Also works with a GIF animation!
st.image("https://upload.wikimedia.org/wikipedia/commons/2/2c/Rotating_earth_%28large%29.gif", caption="Rotating Earth!")

Let's say you have some images in your Snowflake stage, you can stream in the image file and display it with Streamlit.

In [ ]:
create or replace stage NOTEBOOK;

In [ ]:
from snowflake.snowpark.context import get_active_session
session = get_active_session()
session.file.put('snowflake_logo.png',
                 '@NOTEBOOK/Visual_Data_Stories_with_Snowflake_Notebooks',
                 auto_compress=False,
                 overwrite=True)

In [ ]:
LS @NOTEBOOK;

In [ ]:
# Add a query tag to the session. This helps with debugging and performance monitoring.
session.query_tag = {"origin":"sf_sit-is", "name":"aiml_notebooks_fs_with_dbt", "version":{"major":1, "minor":0}, "attributes":{"is_quickstart":0, "source":"notebook"}}

image=session.file.get_stream("@NOTEBOOK/Visual_Data_Stories_with_Snowflake_Notebooks/snowflake_logo.png", decompress=False).read()
st.image(image)

## Interactive data apps 🕹️

Think of each cell in your Snowflake Notebook as a mini Streamlit app. As you interact with your data app, the relevant cells will get re-executed and the results in your app updates.


In [ ]:
st.markdown("""# Interactive Filtering with Streamlit! :balloon:
Values will automatically cascade down the notebook cells""")
value = st.slider("Move the slider to change the filter value 👇", df.snowfall_inches.min(), df.snowfall_inches.max(), df.snowfall_inches.mean(), step = 0.3 )

In [ ]:
# Filter the table from above using the Streamlit slider
filtered_df = df[df["snowfall_inches"]>value].sort_values("snowfall_inches")
count_df = filtered_df.groupby('region').count()['month'].reset_index()

# Chart the number of months above the average
alt.Chart(count_df, title = f"Months above {np.round(value,2)}\" snowfall by region").mark_bar().encode(
    x= alt.X("region", axis = alt.Axis(labelAngle=0)),
    y=alt.Y("month", title = 'Number of months'),
).properties(width=500,height=300)

# Now it's your turn! 🙌 

Try out Notebooks yourself to build your own data narrative!